# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from pathlib import Path
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline


### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = (
        pd.read_csv("data/movie_reviews_eng.csv")
        .rename(columns={"Movie / TV Series": "Title"})
    )
    df_eng["Original Language"] = "en"

    df_fr = (
        pd.read_csv("data/movie_reviews_fr.csv")
        .rename(columns={"Titre": "Title", "Année": "Year", "Critiques": "Review"})
    )
    df_fr["Original Language"] = "fr"

    df_sp = (
        pd.read_csv("data/movie_reviews_sp.csv")
        .rename(columns={"Título": "Title", "Año": "Year", "Sinopsis": "Synopsis", "Críticas": "Review"})
    )
    df_sp["Original Language"] = "sp"

    df = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)
    df = df[["Title", "Year", "Synopsis", "Review", "Original Language"]]
    return df

df = preprocess_data()


In [3]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
16,La Tour Montparnasse Infernale,2001,Deux employés de bureau incompétents se retrou...,"""Je ne peux pas croire que j'ai perdu du temps...",fr
22,Y tu mamá también,2001,Dos amigos adolescentes (Gael García Bernal y ...,"""Y tu mamá también es una película que se qued...",sp
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",en
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en
27,El Bar,2017,Un grupo de personas quedan atrapadas en un ba...,"""El Bar es una película ridícula y sin sentido...",sp
21,La Casa de Papel,(2017-2021),Esta serie de televisión española sigue a un g...,"""La Casa de Papel es una serie emocionante y a...",sp
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",en
19,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",fr
28,Torrente: El brazo tonto de la ley,1998,"En esta comedia española, un policía corrupto ...","""Torrente es una película vulgar y ofensiva qu...",sp
29,El Incidente,2014,"En esta película de terror mexicana, un grupo ...","""El Incidente es una película aburrida y sin s...",sp


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    if not isinstance(text, str) or not text.strip():
        return text

    encoded = tokenizer([text], return_tensors="pt", padding=True, truncation=True)
    generated_tokens = model.generate(**encoded, max_length=512)
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return translated_text[0]


C:\Users\Ohara\AppData\Roaming\Python\Python312\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
# TODO 4: Update the code below

fr_mask = df["Original Language"] == "fr"
es_mask = df["Original Language"] == "sp"

# filter reviews in French and translate to English
fr_reviews = df.loc[fr_mask, "Review"]
fr_reviews_en = fr_reviews.apply(lambda text: translate(text, fr_en_model, fr_en_tokenizer))
df.loc[fr_reviews_en.index, "Review"] = fr_reviews_en

# filter synopsis in French and translate to English
fr_synopsis = df.loc[fr_mask, "Synopsis"]
fr_synopsis_en = fr_synopsis.apply(lambda text: translate(text, fr_en_model, fr_en_tokenizer))
df.loc[fr_synopsis_en.index, "Synopsis"] = fr_synopsis_en

# filter reviews in Spanish and translate to English
es_reviews = df.loc[es_mask, "Review"]
es_reviews_en = es_reviews.apply(lambda text: translate(text, es_en_model, es_en_tokenizer))
df.loc[es_reviews_en.index, "Review"] = es_reviews_en

# filter synopsis in Spanish and translate to English
es_synopsis = df.loc[es_mask, "Synopsis"]
es_synopsis_en = es_synopsis.apply(lambda text: translate(text, es_en_model, es_en_tokenizer))
df.loc[es_synopsis_en.index, "Synopsis"] = es_synopsis_en


In [6]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",sp
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",en
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",en
22,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",sp
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",fr
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",en
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",sp
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",en
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",sp


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [7]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    if not isinstance(text, str) or not text.strip():
        return None

    result = classifier(text)[0]
    label = result["label"].upper()
    return "Negative" if label.startswith("NEG") else "Positive"


Device set to use cpu


In [8]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df["Sentiment"] = df["Review"].apply(lambda text: analyze_sentiment(text, sentiment_classifier))


In [9]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
15,Le Dîner de Cons,1998,The film follows the story of a group of rich ...,"""I didn't like this movie at all. The concept ...",fr,Negative
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",en,Positive
20,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",sp,Positive
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",sp,Negative
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",sp,Negative
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",fr,Positive
26,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",sp,Negative
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",en,Negative
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",fr,Negative
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",sp,Positive


In [10]:
# export the results to a .csv file
Path("result").mkdir(parents=True, exist_ok=True)
output_columns = ["Title", "Year", "Synopsis", "Review", "Sentiment", "Original Language"]
df[output_columns].to_csv("result/reviews_with_sentiment.csv", index=False)
